In [ ]:
# run command 'conda install scikit-image' on project's v-env
# run command 'conda install pandas' on project's v-env

> ## Define Constants
> If DEBUG_TRAIN is true, shows the shape of the tensor during the learning process.<br>
> If DEBUG_DATA is true, shows the data during the learning process.<br>
> MODEL_PATH specifies the path, which models completed learning will be stored.

In [1]:
DEBUG_TRAIN = False
DEBUG_DATA = False
MODEL_PATH = './saved_model/MultiLabelSoftMarginLoss.torchModel'

> ## Define Functions
> **encode_from_utf8**: returns a list of bit that converted from character represented by UTF-8<br>
> **decode_from_bin**: returns a character represented by UTF-8 that converted from list of bit<br>

In [2]:
def encode_from_utf8(x):
    return list(map(float,bin(int(x.encode().hex(),16))[2:]))

In [3]:
def decode_from_bin(x):
    res = ""
    for i in x:
        res+=str(round(i))
        pass
    return bytearray.fromhex(hex(int(res, 2))[2:]).decode()

> ## Define Dataset Class

In [4]:
import os
import torch
import numpy as np
import pandas as pd
from skimage import io, transform
from torch.utils.data import Dataset, DataLoader

class KoreanHandwritingDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.dataset = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = self.dataset.iloc[idx, 0]
        image = io.imread(img_name)
        label = self.dataset.iloc[idx, 1]
        label = np.array([label])
        sample = {'image': image, 'label': label}

        if self.transform:
            sample = self.transform(sample)

        return sample
    pass

> ## Define Transform Class

In [5]:
class ToTensor(object):
    def __call__(self, sample):
        image, label = sample['image'], sample['label']
        # swap color axis because
        # numpy image: H x W x C
        # torch image: C X H X W
        image = image/255.0
        return {'image': torch.from_numpy(image.reshape(1,64,64)),
                'label': torch.tensor(encode_from_utf8(label[0]))}

In [15]:
from torchvision import transforms
csv_file = "./image-data-256/labels-map.csv"
root_dir = "./image-data-256/hangul-images"
batch_size=64
learning_rate=0.02
num_epoch=100

korean_dataset = KoreanHandwritingDataset(csv_file,root_dir, transform=transforms.Compose([ToTensor()]))
#dataloader = DataLoader(korean_dataset, batch_size = batch_size, shuffle = True, num_workers = 0)
dataloader = DataLoader(korean_dataset, batch_size = batch_size, shuffle = False, num_workers = 0)

> ## Define Model

In [7]:
import torch.nn as nn

class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.avgpool1 = nn.AvgPool2d(2,2) # kernel size 2x2 (32 = 64/2)
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(2,2) # kernel size 2x2 (32 = 64/2)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool2d(2,2) # kernel size 2x2 (16 = 32/2)
        self.linear1 = nn.Linear(1152,378)
        self.relu3 = nn.ReLU()
        self.linear2 = nn.Linear(378,128)
        self.relu4 = nn.ReLU()
        self.linear3 = nn.Linear(128,24)
        self.sigmoid = nn.Sigmoid()
        pass
    def forward(self,x):
        out = self.avgpool1(x)
        if DEBUG_TRAIN : print('avgpool1: ', out.shape)
        out = self.conv1(out)
        if DEBUG_TRAIN : print('conv1: ', out.shape)
        out = self.relu1(out)
        if DEBUG_TRAIN : print('relu1: ', out.shape)
        out = self.maxpool1(out)
        if DEBUG_TRAIN : print('maxpool1: ', out.shape)
        out = self.conv2(out)
        if DEBUG_TRAIN : print('conv2: ', out.shape)
        out = self.relu2(out)
        if DEBUG_TRAIN : print('relu2: ', out.shape)
        out = self.maxpool2(out)
        if DEBUG_TRAIN : print('maxpool2: ', out.shape)
        out = out.view(out.size(0),-1)
        if DEBUG_TRAIN : print('view: ', out.shape)
        out = self.linear1(out)
        if DEBUG_TRAIN : print('linear1: ', out.shape)
        out = self.relu3(out)
        if DEBUG_TRAIN : print('relu3: ', out.shape)
        out = self.linear2(out)
        if DEBUG_TRAIN : print('linear2: ', out.shape)
        out = self.relu4(out)
        if DEBUG_TRAIN : print('relu4: ', out.shape)
        out = self.linear3(out)
        if DEBUG_TRAIN : print('linear3: ', out.shape)
        out = self.sigmoid(out)
        if DEBUG_TRAIN : print('sigmoid: ', out.shape)
        if DEBUG_TRAIN : print()
        return out

> ## Create Model
> if saved model exist, load model from disk

In [10]:
import torch.optim as optim
model=CNN().double()
if(os.path.isfile(MODEL_PATH)):
    model.load_state_dict(torch.load(MODEL_PATH))
    model.eval()
    pass
#loss_function=nn.BCEWithLogitsLoss()
loss_function=nn.MultiLabelSoftMarginLoss()
optimizer= torch.optim.Adam(model.parameters(),lr=learning_rate)

> ## Train Model

In [16]:
import matplotlib.pyplot as plt

loss_arr=[]
for i in range(num_epoch):
    for j, sample in enumerate(dataloader):
        x = sample['image']
        y = sample['label']
        optimizer.zero_grad()
        output=model.forward(x)
        loss=loss_function(output,y)
        loss.backward()
        optimizer.step()
        if j == 0:
            if DEBUG_DATA:
                plt.figure(figsize=(32, 32))
                for k, img in enumerate(x):
                    plt.subplot(8, 8, k+1)
                    plt.imshow(img.squeeze())
                    plt.xticks([])
                    plt.yticks([])
                    pass
                plt.show()
            print('epoch: ', i+1, 'sequence: ', j, loss)
            loss_arr.append(loss.detach().numpy())

epoch:  1 sequence:  0 tensor(0.5240, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  2 sequence:  0 tensor(0.5250, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  3 sequence:  0 tensor(0.5247, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  4 sequence:  0 tensor(0.5241, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  5 sequence:  0 tensor(0.5260, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  6 sequence:  0 tensor(0.5236, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  7 sequence:  0 tensor(0.5263, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  8 sequence:  0 tensor(0.5269, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  9 sequence:  0 tensor(0.5267, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  10 sequence:  0 tensor(0.5252, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  11 sequence:  0 tensor(0.5264, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  12 sequence:  0 tensor(0.5238, dtype=torch.float64, grad_fn=<MeanB

epoch:  98 sequence:  0 tensor(0.5133, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  99 sequence:  0 tensor(0.5133, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  100 sequence:  0 tensor(0.5133, dtype=torch.float64, grad_fn=<MeanBackward0>)


> ## Save Current Model

In [ ]:
torch.save(model.state_dict(), PATH)

> ## Test Model

In [12]:
with torch.no_grad():
    for j, sample in enumerate(dataloader):
        x = sample['image']
        y = sample['label']
        
        output = model.forward(x)
        for i in range(len(output)):
            print("출력 값: ", decode_from_bin(output[i].tolist()),
                  ", 실제 값: ", decode_from_bin(y[i].tolist()))
            #print("실제 값: ", output[i].tolist(),
            #      ", 출력 값: ", y[i].tolist())

출력 값:  냤 , 실제 값:  다
출력 값:  냤 , 실제 값:  다
출력 값:  냤 , 실제 값:  다
출력 값:  푘 , 실제 값:  하
출력 값:  푘 , 실제 값:  하
출력 값:  푘 , 실제 값:  하
출력 값:  푘 , 실제 값:  하
출력 값:  죀 , 실제 값:  지
출력 값:  죀 , 실제 값:  지
출력 값:  죀 , 실제 값:  지
출력 값:  죀 , 실제 값:  지
출력 값:  왴 , 실제 값:  이
출력 값:  왴 , 실제 값:  이
출력 값:  왴 , 실제 값:  이
출력 값:  왴 , 실제 값:  이
출력 값:  갰 , 실제 값:  기
출력 값:  갰 , 실제 값:  기
출력 값:  갰 , 실제 값:  기
출력 값:  갰 , 실제 값:  기
출력 값:  렬 , 실제 값:  리
출력 값:  렬 , 실제 값:  리
출력 값:  렬 , 실제 값:  리
출력 값:  렬 , 실제 값:  리
출력 값:  가 , 실제 값:  가
출력 값:  가 , 실제 값:  가
출력 값:  가 , 실제 값:  가
출력 값:  가 , 실제 값:  가
출력 값:  사 , 실제 값:  사
출력 값:  사 , 실제 값:  사
출력 값:  사 , 실제 값:  사
출력 값:  사 , 실제 값:  사
출력 값:  쒐 , 실제 값:  자
출력 값:  쒐 , 실제 값:  자
출력 값:  쒐 , 실제 값:  자
출력 값:  쒐 , 실제 값:  자
출력 값:  뀀 , 실제 값:  대
출력 값:  뀀 , 실제 값:  대
출력 값:  뀀 , 실제 값:  대
출력 값:  뀀 , 실제 값:  대
출력 값:  적 , 실제 값:  적
출력 값:  적 , 실제 값:  적
출력 값:  적 , 실제 값:  적
출력 값:  적 , 실제 값:  적
출력 값:  쒴 , 실제 값:  어
출력 값:  쒴 , 실제 값:  어
출력 값:  쒴 , 실제 값:  어
출력 값:  쒴 , 실제 값:  어
출력 값:  쑄 , 실제 값:  아
출력 값:  쑄 , 실제 값:  아
출력 값:  쑄 , 실제 값:  아


출력 값:  퐼 , 실제 값:  피
출력 값:  퐼 , 실제 값:  피
출력 값:  퐼 , 실제 값:  피
출력 값:  트 , 실제 값:  편
출력 값:  트 , 실제 값:  편
출력 값:  트 , 실제 값:  편
출력 값:  트 , 실제 값:  편
출력 값:  룤 , 실제 값:  매
출력 값:  룤 , 실제 값:  매
출력 값:  룤 , 실제 값:  매
출력 값:  룤 , 실제 값:  매
출력 값:  근 , 실제 값:  근
출력 값:  근 , 실제 값:  근
출력 값:  근 , 실제 값:  근
출력 값:  근 , 실제 값:  근
출력 값:  뀰 , 실제 값:  터
출력 값:  뀰 , 실제 값:  터
출력 값:  뀰 , 실제 값:  터
출력 값:  뀰 , 실제 값:  터
출력 값:  쓅 , 실제 값:  업
출력 값:  쓅 , 실제 값:  업
출력 값:  쓅 , 실제 값:  업
출력 값:  쓅 , 실제 값:  업
출력 값:  렀 , 실제 값:  버
출력 값:  렀 , 실제 값:  버
출력 값:  렀 , 실제 값:  버
출력 값:  렀 , 실제 값:  버
출력 값:  쀍 , 실제 값:  석
출력 값:  쀍 , 실제 값:  석
출력 값:  쀍 , 실제 값:  석
출력 값:  쀍 , 실제 값:  석
출력 값:  듄 , 실제 값:  들
출력 값:  듄 , 실제 값:  들
출력 값:  듄 , 실제 값:  들
출력 값:  듄 , 실제 값:  들
출력 값:  절 , 실제 값:  절
출력 값:  절 , 실제 값:  절
출력 값:  절 , 실제 값:  절
출력 값:  절 , 실제 값:  절
출력 값:  갰 , 실제 값:  결
출력 값:  갰 , 실제 값:  결
출력 값:  갰 , 실제 값:  결
출력 값:  갰 , 실제 값:  결
출력 값:  쑽 , 실제 값:  약
출력 값:  쑽 , 실제 값:  약
출력 값:  쑽 , 실제 값:  약
출력 값:  쑽 , 실제 값:  약
출력 값:  죁 , 실제 값:  직
출력 값:  죁 , 실제 값:  직
출력 값:  죁 , 실제 값:  직
